# Tic-Tac-Toe Reinforcement Learning

Este notebook tem como objetivo demosntrar o simples treinamento através da técnica de aprendizado por reforço no jogo tic-tae-toe:
- Objetivo: entender intuição por trás de *value functions* e políticas *epsilon-greedy* aplicadas a decisões sequenciais.
- Resultado: rodar e treinar um agente simples que aprende jogando tic-tac-toe e visualizar desempenho.

## Contexto e Intuição

- *Value Function* (V(s)): uma estimativa do valor futuro esperado de estar em um determinado estado (ex.: posição do tabuleiro).
- Atualização incremental (TD-like): V(s) <- V(s) + alpha * (V(s') - V(s)). Intuitivamente, ajustamos expectativas com base no resultado observado.
- Política *epsilon-greedy*: mistura entre explorar (testar novas ações) e explorar (usar o que sabemos ser bom).

Aplicação em negócios: decisões sequenciais com incerteza (inventário, pricing, alocação). O mesmo trade-off exploração/exploração aparece em estratégias de mercado e investimento em inovação.

In [ ]:
# Célula de import e configuração
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

LENGTH = 3

In [ ]:
# Definição das classes (Agent, Environment, Human) e funções auxiliares
class Agent:
    def __init__(self, eps=0.1, alpha=0.5):
        self.eps = eps
        self.alpha = alpha
        self.verbose = False
        self.state_history = []

    def setV(self, V):
        self.V = V

    def set_symbol(self, sym):
        self.sym = sym

    def set_verbose(self, v):
        self.verbose = v

    def reset_history(self):
        self.state_history = []

    def take_action(self, env):
        r = np.random.rand()
        best_state = None

        if r < self.eps:
            possible_moves = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i, j))
            idx = np.random.choice(len(possible_moves))
            next_move = possible_moves[idx]
        else:
            pos2value = {}
            next_move = None
            best_value = -1

            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        env.board[i, j] = self.sym
                        state = env.get_state()
                        env.board[i, j] = 0
                        pos2value[(i, j)] = self.V[state]

                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i, j)

        env.board[next_move[0], next_move[1]] = self.sym

    def update_state_history(self, s):
        self.state_history.append(s)

    def update(self, env):
        reward = env.reward(self.sym)
        target = reward

        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha * (target - self.V[prev])
            self.V[prev] = value
            target = value

        self.reset_history()

class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1
        self.o = 1
        self.winner = None
        self.ended = False
        self.num_states = 3 ** (LENGTH * LENGTH)

    def is_empty(self, i, j):
        return self.board[i, j] == 0

    def reward(self, sym):
        if not self.game_over():
            return 0
        return 1 if self.winner == sym else 0

    def get_state(self):
        k = 0
        h = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i, j] == 0:
                    v = 0
                elif self.board[i, j] == self.x:
                    v = 1
                elif self.board[i, j] == self.o:
                    v = 2
                h += (3 ** k) * v
                k += 1
        return h

    def game_over(self, force_recalculate=False):
        if not force_recalculate and self.ended:
            return self.ended

        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:, j].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        for player in (self.x, self.o):
            if self.board.trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True

            if np.fliplr(self.board).trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True

        if np.all((self.board == 0) == False):
            self.winner = None
            self.ended = True
            return True

        self.winner = None
        return False

    def is_draw(self):
        return self.ended and self.winner is None

    def draw_board(self):
        for i in range(LENGTH):
            print("-------------")
            for j in range(LENGTH):
                print("  ", end="")
                if self.board[i, j] == self.x:
                    print("x ", end="")
                elif self.board[i, j] == self.o:
                    print("o ", end="")
                else:
                    print("  ", end="")
            print("")
        print("-------------")

class Human:
    def __init__(self):
        self.sym = None

    def set_symbol(self, sym):
        self.sym = sym

    def take_action(self, env):
        while True:
            move = input("Insira as coordenadas i, j para o próximo movimento (por exemplo: 0,2): ")
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i, j):
                env.board[i, j] = self.sym
                break

def get_state_hash_and_winner(env, i=0, j=0):
    results = []
    for v in (0, env.x, env.o):
        env.board[i, j] = v
        if j == 2:
            if i == 2:
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.winner
                results.append((state, winner, ended))
            else:
                results += get_state_hash_and_winner(env, i + 1, 0)
        else:
            results += get_state_hash_and_winner(env, i, j + 1)
    return results

def initialV_x(env, state_winner_triples):
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.x:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V

def initialV_o(env, state_winner_triples):
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.o:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V

def play_game(p1, p2, env, draw=False):
    current_player = None
    while not env.game_over():
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1

        if draw:
            if draw == 1 and current_player == p1:
                env.draw_board()
            if draw == 2 and current_player == p2:
                env.draw_board()

        current_player.take_action(env)
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)

    if draw:
        env.draw_board()

    p1.update(env)
    p2.update(env)

    return env

In [ ]:
# Célula de treino com métrica simples e visualização
env = Environment()
state_winner_triples = get_state_hash_and_winner(env)

Vx = initialV_x(env, state_winner_triples)
Vo = initialV_o(env, state_winner_triples)

p1 = Agent(eps=0.1, alpha=0.3)
p2 = Agent(eps=0.1, alpha=0.3)
p1.setV(Vx)
p2.setV(Vo)
p1.set_symbol(env.x)
p2.set_symbol(env.o)

T = 2000
p1_wins = 0
p2_wins = 0
draws = 0
history = []
window = deque(maxlen=100)

for t in range(T):
    if t % 200 == 0:
        print('Iter', t)
    env = Environment()
    env = play_game(p1, p2, env)
    if env.winner == env.x:
        p1_wins += 1
        window.append(1)
    elif env.winner == env.o:
        p2_wins += 1
        window.append(0)
    else:
        draws += 1
        window.append(0)
    history.append(np.mean(window) if len(window) else 0)

print('Treino completo')
print('p1 wins:', p1_wins, 'p2 wins:', p2_wins, 'draws:', draws)

plt.figure(figsize=(8,4))
plt.plot(history)
plt.title('Média móvel (janela=100) de vitórias do p1 ao longo do treino')
plt.xlabel('Partidas')
plt.ylabel('Taxa de vitória (p1)')
plt.grid(True)
plt.show()

In [ ]:
# Demonstração: agente treinado vs agente aleatório (sem aprendizado)
class RandomAgent(Agent):
    def take_action(self, env):
        possible_moves = []
        for i in range(LENGTH):
            for j in range(LENGTH):
                if env.is_empty(i, j):
                    possible_moves.append((i, j))
        idx = np.random.choice(len(possible_moves))
        env.board[possible_moves[idx][0], possible_moves[idx][1]] = self.sym

# Comparar 100 partidas
rand = RandomAgent()
rand.set_symbol(env.o)
rand.setV(Vo)
rand.set_verbose(False)

p1.set_verbose(False)
results = { 'p1':0, 'rand':0, 'draw':0 }
N = 100
for i in range(N):
    game_env = Environment()
    game_env = play_game(p1, rand, game_env)
    if game_env.winner == game_env.x:
        results['p1'] += 1
    elif game_env.winner == game_env.o:
        results['rand'] += 1
    else:
        results['draw'] += 1

print('Resultados vs Random (N=', N, '):', results)

## Discussão (para MBA)

- O agente aprende via atualizações incrementais em estados visitados durante um episódio. Isso é similar a atualizar estimativas de valor de projetos com novas evidências.
- Ajustar `alpha` e `eps` controla velocidade de aprendizagem e propensão a experimentar — análogo a orçamento para testes e exploração de novos produtos.
- Próximos passos práticos: modelar estados e recompensas reais do negócio, simular políticas e estimar custo de experimentação.

## Jogar contra o Agente Treinado

Agora você pode jogar contra o agente treinado! Você será o jogador 'O' e o agente treinado será o jogador 'X'.

**Como jogar:**
- Você faz o primeiro movimento após o agente
- Digite as coordenadas no formato `linha,coluna` (ex: `0,0`, `1,2`)
- Posições válidas são de (0,0) a (2,2)
- O tabuleiro é exibido com 'x' para o agente, 'o' para você, e '  ' para células vazias

In [ ]:
def play_game_with_human(agent, human, env, verbose=False):
    """
    Função para jogar uma partida com um jogador humano.
    O agente escolhe primeiro (X), e o humano joga com O.
    """
    current_player = None
    turn_count = 0

    while not env.game_over():
        # Alternar entre jogadores
        if current_player == agent:
            current_player = human
        else:
            current_player = agent

        # Mostrar tabuleiro antes do movimento
        if current_player == human or (current_player == agent and turn_count == 0):
            print("\n[Tabuleiro]")
            env.draw_board()

        # Jogador atual faz um movimento
        if isinstance(current_player, Human):
            print(f"\n[Sua vez (O)]")
            current_player.take_action(env)
        else:
            print(f"\n[Vez do Agente (X)]")
            agent.take_action(env)

        # Atualizar histórico de estados
        state = env.get_state()
        agent.update_state_history(state)
        human.update_state_history(state) if hasattr(human, 'update_state_history') else None
        turn_count += 1

    # Mostrar tabuleiro final
    print("\n[Tabuleiro Final]")
    env.draw_board()

    # Determinar resultado
    if env.winner == agent.sym:
        print("\n AGENTE VENCEU!")
    elif env.winner == human.sym:
        print("\n VOCÊ VENCEU!")
    else:
        print("\n EMPATE!")

    return env

# Preparar para jogar
print("Agente treinado pronto para jogar!")
print("Você joga com 'O', o agente joga com 'X'")

In [ ]:
# Jogar partidas com o usuário
human = Human()
human.set_symbol(env.o)
p1.set_verbose(False)

# Loop para jogar múltiplas partidas
play_again = True
game_count = 0
human_wins = 0
agent_wins = 0
game_draws = 0

while play_again:
    game_count += 1
    print(f"\n=== PARTIDA {game_count} ===")
    
    game_env = Environment()
    final_env = play_game_with_human(p1, human, game_env)
    
    # Registrar resultado
    if final_env.winner == final_env.x:
        agent_wins += 1
    elif final_env.winner == final_env.o:
        human_wins += 1
    else:
        game_draws += 1
    
    # Perguntar se deseja jogar novamente
    response = input("\nDeseja jogar novamente? (s/n): ").strip().lower()
    if response != 's' and response != 'sim':
        play_again = False

# Exibir placar final
print(f"\n=== PLACAR FINAL ===")
print(f"Você (O): {human_wins} vitórias")
print(f"Agente (X): {agent_wins} vitórias")
print(f"Empates: {game_draws}")
print(f"Total de partidas: {game_count}")